In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.types import Row, StringType

In [3]:
def rdd_to_hdfs(rdd):
  if not rdd.isEmpty():
    #print(rdd.collect())
    spark=SparkSession.builder.appName("App").getOrCreate()
    #row_rdd=rdd.flatmap(lambda x:Row(x))
    l=[]

    lst=rdd.collect()
    for s in lst:
      l.append(str(s).split(","))

    schema=""
    for i in range(len(l[0])):
      schema=schema+(chr(ord('`')+(i+1)))+" string, "

    schema=schema[0:len(schema)-2]

    #print(schema)

    df=spark.createDataFrame(l, schema=schema)
    df.show()

In [4]:
# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext("local[2]")
ssc = StreamingContext(sc, 1)

# Create a DStream that will connect to hostname:port, like localhost:9999
lines = ssc.socketTextStream("localhost", 9999)

batch=lines.map(lambda line: line.split("\n"))
#words=batch_size.flatMap(lambda line: line.split(","))

#words.pprint()

batch.foreachRDD(rdd_to_hdfs)

ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

25/08/16 18:55:45 WARN Utils: Your hostname, dsbda-vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/08/16 18:55:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/16 18:55:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/16 18:55:47 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/08/16 18:55:47 WARN BlockManager: Block input-0-1755363347600 replicated to only 0 peer(s) instead of 1 peers
25/08/16 18:55:48 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/08/16 18:55:48 WARN BlockManager: Block input-0-1755363347800 replicated to only 0 peer(s) instead of 1 peers
25

+------+--------------------+--------------------+-------+-------------+----------------+------+------+----------+--------+------------+--------------------+
|     a|                   b|                   c|      d|            e|               f|     g|     h|         i|       j|           k|                   l|
+------+--------------------+--------------------+-------+-------------+----------------+------+------+----------+--------+------------+--------------------+
|["['1'| 'Harry Potter an...| 'J.K. Rowling/Ma...| '4.57'| '0439785960'| '9780439785969'| 'eng'| '652'| '2095690'| '27591'| '9/16/2006'| 'Scholastic Inc....|
+------+--------------------+--------------------+-------+-------------+----------------+------+------+----------+--------+------------+--------------------+



25/08/16 18:55:51 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/08/16 18:55:51 WARN BlockManager: Block input-0-1755363351000 replicated to only 0 peer(s) instead of 1 peers


+------+--------------------+--------------------+-------+-------------+----------------+------+------+----------+--------+------------+--------------------+
|     a|                   b|                   c|      d|            e|               f|     g|     h|         i|       j|           k|                   l|
+------+--------------------+--------------------+-------+-------------+----------------+------+------+----------+--------+------------+--------------------+
|["['2'| 'Harry Potter an...| 'J.K. Rowling/Ma...| '4.49'| '0439358078'| '9780439358071'| 'eng'| '870'| '2153167'| '29221'|  '9/1/2004'| 'Scholastic Inc....|
|["['4'| 'Harry Potter an...|      'J.K. Rowling'| '4.42'| '0439554896'| '9780439554893'| 'eng'| '352'|    '6333'|   '244'| '11/1/2003'|     'Scholastic']"]|
+------+--------------------+--------------------+-------+-------------+----------------+------+------+----------+--------+------------+--------------------+



25/08/16 18:55:51 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/08/16 18:55:51 WARN BlockManager: Block input-0-1755363351400 replicated to only 0 peer(s) instead of 1 peers
25/08/16 18:55:51 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/08/16 18:55:51 WARN BlockManager: Block input-0-1755363351600 replicated to only 0 peer(s) instead of 1 peers


...

25/08/16 18:56:17 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/08/16 18:56:17 WARN BlockManager: Block input-0-1755363377400 replicated to only 0 peer(s) instead of 1 peers
25/08/16 18:56:18 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/08/16 18:56:18 WARN BlockManager: Block input-0-1755363377800 replicated to only 0 peer(s) instead of 1 peers
25/08/16 18:56:18 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/08/16 18:56:18 WARN BlockManager: Block input-0-1755363378000 replicated to only 0 peer(s) instead of 1 peers


+--------+--------------------+--------------------+-------+-------------+----------------+------+------+--------+-------+------------+-----------------+
|       a|                   b|                   c|      d|            e|               f|     g|     h|       i|      j|           k|                l|
+--------+--------------------+--------------------+-------+-------------+----------------+------+------+--------+-------+------------+-----------------+
|["['231'| 'I am Charlotte ...|         'Tom Wolfe'| '3.42'| '0312424442'| '9780312424442'| 'eng'| '738'| '20888'| '1688'| '8/30/2005'| 'Picador USA']"]|
|["['237'| 'Poetry for Youn...| 'Edward Lear/Lau...| '3.91'| '0806930772'| '9780806930770'| 'eng'|  '48'|    '66'|   '19'| '10/1/2001'|    'Sterling']"]|
+--------+--------------------+--------------------+-------+-------------+----------------+------+------+--------+-------+------------+-----------------+



25/08/16 18:56:18 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/08/16 18:56:18 WARN BlockManager: Block input-0-1755363378400 replicated to only 0 peer(s) instead of 1 peers
25/08/16 18:56:18 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/08/16 18:56:18 WARN BlockManager: Block input-0-1755363378600 replicated to only 0 peer(s) instead of 1 peers
25/08/16 18:56:19 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/08/16 18:56:19 WARN BlockManager: Block input-0-1755363378800 replicated to only 0 peer(s) instead of 1 peers
ERROR:root:Exception while sending command.                         (0 + 1) / 1]
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=59>

During handling of the above except

Py4JError: An error occurred while calling o23.awaitTermination

+--------+--------------------+--------------------+-------+-------------+----------------+------+------+--------+-------+------------+--------------------+
|       a|                   b|                   c|      d|            e|               f|     g|     h|       i|      j|           k|                   l|
+--------+--------------------+--------------------+-------+-------------+----------------+------+------+--------+-------+------------+--------------------+
|["['244'| 'The Puffin Book...|     'Quentin Blake'| '4.02'| '0140366601'| '9780140366600'| 'eng'| '287'|    '81'|    '6'| '10/3/1996'|         'Puffin']"]|
|["['245'| 'Henry Miller on...| 'Henry Miller/Th...| '4.22'| '0811201120'| '9780811201124'| 'eng'| '217'|   '981'|   '52'|  '2/1/1964'| 'New Directions ...|
|["['247'| 'Quiet Days in C...|      'Henry Miller'| '3.69'| '080213016X'| '9780802130167'| 'eng'| '154'|  '3381'|  '141'| '1/13/1994'|    'Grove Press']"]|
|["['249'|  'Tropic of Cancer'| 'Henry Miller/Ji...| '3.68

In [ ]:
sc.stop()